## Кадры решают всё (но это не точно)

### Кадры, да...

In [4]:
%%time

# импорт библиотек
import pandas as pd
import numpy as np
import os
import sys
import shutil  # упаковка в архив
import warnings

warnings.simplefilter(action="ignore")

# путь к безликим
unpersonal_path = 'C:/personal_auto/unpersonal/'

# список файлов в целевой директории
source_files_list = os.listdir(unpersonal_path)


CPU times: total: 0 ns
Wall time: 2 ms


In [8]:
df_dt = pd.DataFrame()
for file in source_files_list:
    df_tmp = pd.read_csv(
        unpersonal_path + file,
        sep=";",
        encoding="ansi",
    )
    df_dt = pd.concat([df_dt, df_tmp])

In [14]:
df_dt = df_dt.replace(",", ".", regex=True).apply(pd.to_numeric, errors="ignore")

In [19]:
df_dt['3. Адрес МО. Наим. субъекта РФ'] = df_dt['3. Адрес МО. Наим. субъекта РФ'].fillna('не определён')

#### Поиск и замена регионов, имеющих различные наименования

In [20]:
df_regions = pd.DataFrame(df_dt['3. Адрес МО. Наим. субъекта РФ']).drop_duplicates().reset_index(drop=True)

In [24]:
df_regions.head(50)

,3. Адрес МО. Наим. субъекта РФ
0,не определён
1,Алтайский край
2,Амурская область
3,Архангельская область
4,Астраханская область
5,Белгородская область
6,Брянская область
7,Владимирская область
8,Волгоградская область
9,Вологодская область


In [42]:
# сравнение текстов - не поможет, к сожалению
import difflib

def similarity(s1, s2):
  """сравнение текстов"""
  normalized1 = s1.lower()
  normalized2 = s2.lower()
  matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
  return matcher.ratio()

In [3]:
s1 = 'Краснодартский край'
s2 = 'Красноярский край'
print(similarity(s1, s2))

0.8888888888888888


In [26]:
def words(string):
    """преобразование строки в список, элементами которого являются
    части строки, разбитой по неалфавитным символам. Все элементы списка
    приведены к нижнему регистру"""
    str = ''
    for symbol in string:
        symbol = symbol if symbol.isalpha() else '$'
        str = str + symbol
    lst = str.split('$')
    while '' in lst:
        lst.remove('')
    return [element.lower() for element in lst]

In [51]:
s_1 = 'Луганская Народная Республика'
s_2 = 'Донецкая Народная Республика'
result_1 = words(s_1)
result_2 = words(s_2)

In [52]:
lst = result_1 + result_2
ratio = len(set(lst))/len(lst)
ratio

0.6666666666666666

In [53]:
similarity(s_1, s_2)

0.8421052631578947

1.0 - совсем разные   
0.75 - один тип субъекта РФ   
0.6 - есть что-то общее   
если ratio < 0.60 - это один субъект   

In [137]:
result_1

['татарская', 'республика', 'татарстан']

In [ ]:
# проверяем количество файлов в целевой директории
if len(source_file) == 0:
    print("Нет целевого файла, программа завершает работу")
    sys.exit()
elif len(source_file) > 1:
    print(f"В целевой директории {len(source_file)} файла(ов)")
    # тогда будем использовать файл с максимальным временем модификации
    # словарь для хранения времени модификации (ключ) файлов (значение)
    dict_ = {}
    # список времён модификации файлов
    list_ = []
    for file in source_file:
        path = soutce_path + file
        mtime = os.path.getmtime(path)
        dict_[mtime] = file
        list_.append(mtime)
    list_.sort(reverse=True)
    source_file_result = dict_[list_[0]]
    print(
        f"Для работы используется файл {source_file_result}, имеющий наиболее позднее время модификации\nЕсли это не то, что вам нужно, измените содержимое целевой директории"
    )
else:
    source_file_result = source_file[0]
    print(f"В целевой директории единственный файл {source_file_result}")

# путь к целевому файлу
soutce_file_path = soutce_path + source_file_result

# префикс для имён файлов-результатов
pref = source_file_result.split(".")[0]

# полный путь к итоговому экселевскому файлу ФЛК-1,2
result_12_exc_path = result_12_exc_path + pref + "_flk12_result.xlsx"
# полный путь к итоговому файлу ФЛК-3
result_3_path = result_3_path + pref + "_flk3_result.csv"
# полный путь к архиву
result_12_files_path_arch = result_12_files_path_arch + pref + "_flk12_result"


# ФЛК-1, 2

# данные для фильтрации исходного файла
last_status = ["На согласовании", "На редактировании (после отклонения)", "Запрос на отклонение", "Согласовано"]
# last_status = ["На согласовании", "На редактировании (после отклонения)"]
last_status_date = "2020-07-10"  # сначала год, месяц, потом число. Отчётный период начинается в 00:00 указанной даты
next_status_date = "2073-07-03"  # и продолжается до 23:59 даты перед конечной

print(f"Статус: {last_status}")
print(f"Дата начала: {last_status_date}")
print(f"Дата окончания: {next_status_date}")

# сделать пустые фреймы для всех фреймов ошибок для корректной выгруки в Excel

# удаление всех фреймов, оставшихся (вдруг) от предыдущей работы скрипта
try:
    del df_notation
except Exception:
    pass
try:
    del df_ass_0
except Exception:
    pass
try:
    del df_ass
except Exception:
    pass
try:
    del df_ass_success
except Exception:
    pass
try:
    del df_ass_error
except Exception:
    pass
try:
    del df_mo_error
except Exception:
    pass
try:
    del df_dept_error
except Exception:
    pass
try:
    del df_ass_erorr_sum
except Exception:
    pass
try:
    del df_mo_count_more_save
except Exception:
    pass
try:
    del df_ass_mo_plan_zero
except Exception:
    pass
try:
    del df_ass_mo_fact_zero
except Exception:
    pass


# пустые фреймы для будущих ошибок (без них вылетает сохранение в excel, когда нет ошибок)
df_dept_error = pd.DataFrame(
    columns=[
        "id СП",
        "Регион",
        "наименование оборудования",
        "непосредственно заявка СП",
        "сумма заявок по кабинетам",
    ]
)
df_mo_error = pd.DataFrame(
    columns=[
        "id МО",
        "Регион",
        "наименование оборудования",
        "непосредственно заявка МО",
        "сумма заявок по СП",
    ]
)
df_ass_mo_plan_zero = pd.DataFrame()
df_ass_mo_fact_zero = pd.DataFrame()

# загрузка исходного файла - путь изменён!
df_ass = pd.read_csv(
    soutce_file_path,
    sep=";",
    header=1,
    encoding="utf-8-sig",
)

# проверка наличия новых столбцов
if (
    df_ass.columns[1] != "Текущий статус"
    or df_ass.columns[2] != "Дата последнего статуса"
):
    print("Неверная структура исходных данных")
else:
    print("Структура исходных данных верная")

# преобразование формата даты для фильтрации
# df_ass["Дата последнего статуса"] = df_ass["Дата последнего статуса"].astype("datetime64[ns]", format = "%d.%m.%Y %H:%M:%S")
df_ass["Дата последнего статуса"] = pd.to_datetime(
    df_ass["Дата последнего статуса"], format="%d.%m.%Y %H:%M:%S"
)

# копия исходника для потомков
df_ass_0 = df_ass.copy()

# фильтрация по статусу и последней дате, удаление столбцов статуса и последней даты
df_ass = df_ass[df_ass["Текущий статус"].isin(last_status)]
df_ass = df_ass[df_ass["Дата последнего статуса"] >= last_status_date]
df_ass = df_ass[df_ass["Дата последнего статуса"] < next_status_date]
df_ass = df_ass.reset_index(drop=True)
# сначала сохраним в файл для контроля
df_ass.to_csv(
    result_12_files_path + "/исходные_данные_до_удаления_столбцов.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)

cols = [1, 2]
df_ass = df_ass.drop(df_ass.columns[cols], axis=1)
# теперь получаем нормальный исходник, с которым работали до с самого начала, а не вот это вот всё
# дальше работаем так, словно ничего в исходнике не менялось


# очистка полностю пустых строк
row_count_0 = df_ass.shape[0]
df_ass = df_ass.dropna(how="all")
print(f"Удалено {row_count_0 - df_ass.shape[0]} полностью пустых строк")

# заполнение пустых значений вниз
df_ass[df_ass.columns[0]] = df_ass[[df_ass.columns[0]]].ffill(axis=0)
df_ass[df_ass.columns[6]] = df_ass[[df_ass.columns[6]]].ffill(axis=0)
df_ass[df_ass.columns[7]] = df_ass[[df_ass.columns[7]]].ffill(axis=0)

# замена пропусков на 0
df_ass = df_ass.fillna(0)

# сохранение результата в файл
df_ass.to_csv(
    result_12_files_path + "/исходные_данные(.csv).csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)

# добавление столбца уровня лечебницы
df_ass = df_ass.astype({df_ass.columns[1]: "str"})
df_ass["lvl"] = df_ass.apply(lambda x: len(x[df_ass.columns[1]].split("_")) - 2, axis=1)

# переименование столбца с некошерным названием
df_ass = df_ass.rename(columns={df_ass.columns[1]: "ID"})

# разделение на фреймы МО, СП и кабинетов, удаление ненужных данных
df_ass_mo = df_ass[df_ass["lvl"] == 0].reset_index(drop=True)
df_ass_dept = df_ass[df_ass["lvl"] == 1].reset_index(drop=True)
df_ass_room = df_ass[df_ass["lvl"] == 2].reset_index(drop=True)
df_ass_mo.drop(
    [
        df_ass.columns[0],
        df_ass.columns[2],
        df_ass.columns[3],
        df_ass.columns[4],
        df_ass.columns[5],
        df_ass.columns[6],
        df_ass.columns[7],
        df_ass.columns[8],
        "lvl",
    ],
    axis=1,
    inplace=True,
)
df_ass_dept.drop(
    [
        df_ass.columns[0],
        df_ass.columns[2],
        df_ass.columns[3],
        df_ass.columns[4],
        df_ass.columns[5],
        df_ass.columns[6],
        df_ass.columns[7],
        df_ass.columns[8],
        "lvl",
    ],
    axis=1,
    inplace=True,
)
df_ass_room.drop(
    [
        df_ass.columns[0],
        df_ass.columns[2],
        df_ass.columns[3],
        df_ass.columns[4],
        df_ass.columns[5],
        df_ass.columns[6],
        df_ass.columns[7],
        df_ass.columns[8],
        "lvl",
    ],
    axis=1,
    inplace=True,
)

# а что, если в фрейме МО какие-то лечебницы встречаются более одного раза?
df_mo_count = pd.DataFrame(df_ass_mo["ID"].value_counts()).reset_index()
df_mo_count_more = df_mo_count[df_mo_count["count"] > 1]
df_mo_count_more_save = df_mo_count_more.merge(
    df_ass, left_on="ID", right_on="ID", how="left"
)  # для сохранения
# удалим из сохраняемого фрейма столбцы, заполненные в данном случае - при повторяющихся кодах лечебниц - некорректно
df_mo_count_more_save = df_mo_count_more_save.drop(
    [
        "Регион",
        "Численность прикрепленного населения",
        "Численность обслуживаемого населения ",
    ],
    axis=1,
)
if len(df_mo_count_more["ID"].tolist()) > 0:
    print("Внимание! В исходных данных повторяются следующие МО:")
    print(*df_mo_count_more["ID"].tolist(), sep="\n")
else:
    print("Коды МО в исходных данных не повторяются")

# а что ести какие-то фреймы пусты?
flag = 0  # показатель уровня иерархии
if df_ass_dept.empty:
    flag = 1
    print("Иерархия данных отсутствует, представлены только МО")
if df_ass_room.empty:
    flag = 2
    print("Представлены данные по МО и подразделениям, кабинетов нет")


if flag == 0:  # есть и МО, и СП, и кабинеты
    print("Структура данных полная, есть и МО, и СП, и кабинеты")
    # добавление в фреймы СП и кабинетов id более выского уровня иерархии
    df_ass_dept.insert(
        1, "id", df_ass_dept.apply(lambda x: "_".join(x["ID"].split("_")[:-1]), axis=1)
    )
    df_ass_room.insert(
        1, "id", df_ass_room.apply(lambda x: "_".join(x["ID"].split("_")[:-1]), axis=1)
    )

    # группировка фреймов СП и кабинетов по более высокому уровню иерархии
    df_ass_dept_group_mo = df_ass_dept.groupby("id").sum().reset_index()
    df_ass_dept_group_room = df_ass_dept.groupby("ID").sum().reset_index()
    df_ass_room_group = df_ass_room.groupby("id").sum().reset_index()
    df_ass_dept_group_mo.drop(["ID"], axis=1, inplace=True)
    df_ass_dept_group_room.drop(["id"], axis=1, inplace=True)
    df_ass_room_group.drop(["ID"], axis=1, inplace=True)

    # добавление данных об источнике
    df_ass_mo.insert(1, "source", "mo")
    df_ass_mo = df_ass_mo.rename(columns={"ID": "id"})
    df_ass_dept_group_mo.insert(1, "source", "dept")
    df_ass_dept_group_room.insert(1, "source", "dept")
    df_ass_dept_group_room = df_ass_dept_group_room.rename(columns={"ID": "id"})
    df_ass_room_group.insert(1, "source", "room")

    # соединение фреймов
    df_mo = (
        pd.concat([df_ass_mo, df_ass_dept_group_mo])
        .sort_values(["id"], ascending=False)
        .reset_index(drop=True)
    )
    df_dept = (
        pd.concat([df_ass_dept_group_room, df_ass_room_group])
        .sort_values(["id"], ascending=False)
        .reset_index(drop=True)
    )

    # формирование спиков кодов лечебниц строк с повторяющимися кодами лечебниц
    mo_index_list = []
    for index in range(0, df_mo.shape[0] - 1):
        if df_mo.loc[index]["id"] == df_mo.loc[index + 1]["id"]:
            mo_index_list.append(df_mo.loc[index]["id"])

    dept_index_list = []
    for index in range(0, df_dept.shape[0] - 1):
        if df_dept.loc[index]["id"] == df_dept.loc[index + 1]["id"]:
            dept_index_list.append(df_dept.loc[index]["id"])

    # удаление непарных строк
    df_mo_result = df_mo.loc[df_mo["id"].isin(mo_index_list)]
    df_dept_result = df_dept.loc[df_dept["id"].isin(dept_index_list)]
    for column in df_mo_result.columns:
        if df_mo_result[column].sum() == 0:
            df_mo_result = df_mo_result.drop(column, axis=1)
    df_mo_result = df_mo_result.reset_index(drop=True)

    for column in df_dept_result.columns:
        if df_dept_result[column].sum() == 0:
            df_dept_result = df_dept_result.drop(column, axis=1)
    df_dept_result = df_dept_result.reset_index(drop=True)

elif flag == 2:  # нет кабинетов, только МО и СП
    df_dept_result = pd.DataFrame()  # пустой фрейм для единообразия
    df_ass_dept.insert(
        1, "id", df_ass_dept.apply(lambda x: "_".join(x["ID"].split("_")[:-1]), axis=1)
    )

    # группировка фреймов СП и кабинетов по более высокому уровню иерархии
    df_ass_dept_group_mo = df_ass_dept.groupby("id").sum().reset_index()
    df_ass_dept_group_mo.drop(["ID"], axis=1, inplace=True)

    # добавление данных об источнике
    df_ass_mo.insert(1, "source", "mo")
    df_ass_mo = df_ass_mo.rename(columns={"ID": "id"})
    df_ass_dept_group_mo.insert(1, "source", "dept")

    # соединение фреймов
    df_mo = (
        pd.concat([df_ass_mo, df_ass_dept_group_mo])
        .sort_values(["id"], ascending=False)
        .reset_index(drop=True)
    )

    # формирование списков кодов лечебниц строк с повторяющимися кодами лечебниц
    mo_index_list = []
    for index in range(0, df_mo.shape[0] - 1):
        if df_mo.loc[index]["id"] == df_mo.loc[index + 1]["id"]:
            mo_index_list.append(df_mo.loc[index]["id"])

    # удаление непарных строк
    df_mo_result = df_mo.loc[df_mo["id"].isin(mo_index_list)]
    for column in df_mo_result.columns:
        if df_mo_result[column].sum() == 0:
            df_mo_result = df_mo_result.drop(column, axis=1)
    df_mo_result = df_mo_result.reset_index(drop=True)
else:
    print("Иерархия отсутстствует, нет предмета для проверки")

# удалим из исходных данных не нужный уже и вредный по сути столбец lvl
df_ass = df_ass.drop("lvl", axis=1)

# собственно проверка соответствия количества оборудования

# сделаем маленький кусочек исходного фрейма для присоединения к результатам
df_ass_l = df_ass[["Регион", "ID", "Наименование МО"]]

# проверка df_mo_result
mo_error = []
for mo in df_mo_result["id"].unique():
    df_test_mo = df_mo_result[df_mo_result["id"] == mo]
    # проверка на количество строк в df_test_mo
    # если строк не 2, вся информация сохраняется в файл и проверка такой МО не осуществляется
    if df_test_mo.shape[0] != 2 or len(df_test_mo["source"].unique()) != 2:
        path = result_12_files_path + "/" + mo + ".csv"
        df_test_mo.to_csv(path, sep=";", encoding="utf-8-sig", index=False)
    else:
        for column in df_test_mo.columns[2:]:
            if (
                df_test_mo.loc[df_test_mo["source"] == "mo"][column].values[0]
                < df_test_mo.loc[df_test_mo["source"] == "dept"][column].values[0]
            ):
                mo_error.append(mo)
                mo_error.append(column)
                mo_error.append(
                    df_test_mo.loc[df_test_mo["source"] == "mo"][column].values[0]
                )
                mo_error.append(
                    df_test_mo.loc[df_test_mo["source"] == "dept"][column].values[0]
                )
mo_error_list = [mo_error[i : i + 4] for i in range(0, len(mo_error), 4)]
df_mo_error = pd.DataFrame(
    mo_error_list,
    columns=[
        "id МО",
        "наименование оборудования",
        "непосредственно заявка МО",
        "сумма заявок по СП",
    ],
)

# присоединение данных из исходного фрейма
try:
    df_mo_error = df_mo_error.merge(
        df_ass_l, left_on="id МО", right_on="ID", how="left"
    )
    df_mo_error = df_mo_error[
        [
            "id МО",
            "Регион",
            "Наименование МО",
            "наименование оборудования",
            "непосредственно заявка МО",
            "сумма заявок по СП",
        ]
    ]
except Exception:
    print("Трабл при присоединении названия региона в ошибках МО")

# сохранение результата в файл
if df_mo_error.empty:
    print("Расхождений данных между МО и СП нет")
    df_mo_error = pd.DataFrame(
        columns=[
            "id МО",
            "наименование оборудования",
            "непосредственно заявка МО",
            "сумма заявок по СП",
        ]
    )
else:
    df_mo_error.to_csv(
        result_12_files_path + "/несоответствие_МО-СП.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Несоответствие между МО и СП сохранены в файл")

# проверка df_dept_result
if df_dept_result.empty:
    pass
else:
    dept_error = []
    for mo in df_dept_result["id"].unique():
        df_test_dept = df_dept_result[df_dept_result["id"] == mo]
        # проверка на количество строк в df_test_dept
        # если строк не 2, вся информация сохраняется в файл и проверка такого подразделения не осуществляется
        if df_test_dept.shape[0] != 2 or len(df_test_dept["source"].unique()) != 2:
            path = result_12_files_path + "/" + mo + "_dept.csv"
            df_test_dept.to_csv(path, sep=";", encoding="utf-8-sig", index=False)
        else:
            for column in df_test_dept.columns[2:]:
                if (
                    df_test_dept.loc[df_test_dept["source"] == "room"][column].values[0]
                    > df_test_dept.loc[df_test_dept["source"] == "dept"][column].values[
                        0
                    ]
                ):
                    dept_error.append(mo)
                    dept_error.append(column)
                    dept_error.append(
                        df_test_dept.loc[df_test_dept["source"] == "dept"][
                            column
                        ].values[0]
                    )
                    dept_error.append(
                        df_test_dept.loc[df_test_dept["source"] == "room"][
                            column
                        ].values[0]
                    )
                    # print(mo, column, df_test_dept.loc[df_test_dept['source']=='room'][column].values[0], df_test_dept.loc[df_test_dept['source']=='dept'][column].values[0])
    dept_error_list = [dept_error[i : i + 4] for i in range(0, len(dept_error), 4)]
    df_dept_error = pd.DataFrame(
        dept_error_list,
        columns=[
            "id СП",
            "наименование оборудования",
            "непосредственно заявка СП",
            "сумма заявок по кабинетам",
        ],
    )
    try:
        df_dept_error = df_dept_error.merge(
            df_ass_l, left_on="id СП", right_on="ID", how="left"
        )
        df_dept_error = df_dept_error[
            [
                "id СП",
                "Регион",
                "Наименование МО",
                "наименование оборудования",
                "непосредственно заявка СП",
                "сумма заявок по кабинетам",
            ]
        ]
    except Exception:
        print("Трабл при присоединении названия региона в ошибках СП")

    # сохранение результата в файл
    if df_dept_error.empty:
        df_dept_error = pd.DataFrame(
            columns=[
                "id СП",
                "наименование оборудования",
                "непосредственно заявка СП",
                "сумма заявок по кабинетам",
            ]
        )
    else:
        df_dept_error.to_csv(
            result_12_files_path + "/несоответствие_СП-кабинеты.csv",
            sep=";",
            encoding="utf-8-sig",
            index=False,
        )
        print("Несоответствия между СП и кабинетами сохранены в файл")

# годная для дальнейшей работы структура фреймов с ошибками, если ошибок не было (котстыль)
if df_dept_error.shape[0] == 0:
    df_dept_error = pd.DataFrame(
        columns=[
            "id СП",
            "Регион",
            "наименование оборудования",
            "непосредственно заявка СП",
            "сумма заявок по кабинетам",
        ]
    )
if df_mo_error.shape[0] == 0:
    df_mo_error = pd.DataFrame(
        columns=[
            "id МО",
            "Регион",
            "наименование оборудования",
            "непосредственно заявка МО",
            "сумма заявок по СП",
        ]
    )

# проверка на заполненность данных - ищем регионы, которые не попросили вообще ничего
# группировка и суммирование количества оборудования по регионам
cols = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
]  # столбцы для удаления (иначе трабл с суммированием по ненужным столбцам, где есть и буквы, и цифры)
df_ass_regions = df_ass.drop(df_ass.columns[cols], axis=1)
df_ass_regions_sum = df_ass_regions.groupby("Регион").sum().reset_index()
# добавляем столбец с суммой заявок по регионам
df_ass_regions_sum["region_sum"] = df_ass_regions_sum[list(df_ass_regions_sum)[1:]].sum(
    axis=1
)
# отбираем успешные и плохие регионы по сумме заявленного и формируем списки регионов
df_ass_success_sum = df_ass_regions_sum[
    df_ass_regions_sum["region_sum"] != 0
].reset_index()
df_ass_erorr_sum = df_ass_regions_sum[
    df_ass_regions_sum["region_sum"] == 0
].reset_index()
bad_regions_sum_set = set(df_ass_erorr_sum["Регион"].tolist())
bad_regions_sum_list = list(bad_regions_sum_set)
df_ass_erorr_sum = df_ass.loc[df_ass["Регион"].isin(bad_regions_sum_list)]

# проверка на заполненность данных 2 - ищем регионы с нулевым планом или нулевым фактом
# удаляем непотребные столбцы
cols = [
    2,
    3,
    4,
    5,
    6,
    7,
    8,
]  # столбцы для удаления (иначе трабл с суммированием по ненужным столбцам, где есть и буквы, и цифры)
df_ass_mo_regions = df_ass.drop(df_ass.columns[cols], axis=1)
# разделяем строки на "план" и "факт"
# удаляем столбцы "по факту"
del_columns_list_fact = []
for col in df_ass_mo_regions.columns:
    if col.split()[-1] == "факту":
        del_columns_list_fact.append(col)
df_ass_mo_plan = df_ass_mo_regions.drop(del_columns_list_fact, axis=1)
# удаляем столбцы "по плану"
del_columns_list_plan = []
for col in df_ass_mo_regions.columns:
    if col.split()[-1] == "плану":
        del_columns_list_plan.append(col)
df_ass_mo_fact = df_ass_mo_regions.drop(del_columns_list_plan, axis=1)
# добавим столбцы с суммой плана и факта по каждой строке
df_ass_mo_plan["count"] = df_ass_mo_plan.iloc[:, 2:].sum(axis=1)
df_ass_mo_fact["count"] = df_ass_mo_fact.iloc[:, 2:].sum(axis=1)
# отберём регионы с нулевым планом и нулевым фактом
df_ass_mo_plan_zero = df_ass_mo_plan[df_ass_mo_plan["count"] == 0]
df_ass_mo_fact_zero = df_ass_mo_fact[df_ass_mo_fact["count"] == 0]
# плохие регионы по сумме плана
bad_regions_sum_plan_set = set(df_ass_mo_plan_zero["Регион"].tolist())
bad_regions_sum_plan_list = list(bad_regions_sum_plan_set)
# плохие регионы по сумме факта
bad_regions_sum_fact_set = set(df_ass_mo_fact_zero["Регион"].tolist())
bad_regions_sum_fact_list = list(bad_regions_sum_set)
# если есть плохие регионы по сумме плана и факта, выгрузим их в файл
if df_ass_mo_plan_zero.empty:
    print("Регионов с нулевым планом нет")
    bad_regions_sum_plan_set = set()
else:
    df_ass_mo_plan_zero.to_csv(
        result_12_files_path + "/пустая_сумма_плана.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Данные по регионам с нулевой суммой плана сохранены в файл")
if df_ass_mo_fact_zero.empty:
    print("Регионов с нулевым фактом нет")
    bad_regions_sum_fact_set = set()
else:
    df_ass_mo_fact_zero.to_csv(
        result_12_files_path + "/пустая_сумма_факта.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Данные по регионам с нулевой суммой факта сохранены в файл")

# Антон велел убрать из успешных регионы с пустым планом ИЛИ пустой суммой. Но я это делать пока не буду, просто сохраню такие регионы
bad_regions_plan_fact_set = bad_regions_sum_fact_set.union(
    bad_regions_sum_plan_set
)  # множество плохих регионов по нулевому плану ИЛИ нулевому факту - нигде не используется!

# если плохие регионы есть, выгрузим их в файл
if df_ass_erorr_sum.empty:
    print("Регионов с полностью пустой заявкой нет")
else:
    df_ass_erorr_sum.to_csv(
        result_12_files_path + "/пустая_сумма_заявок.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Данные по регионам с нулевой суммой сохранены в файл")

# считаем ошибкой заявки ТОЛЬКО ПУСТОЙ ПЛАН: bad_regions_sum_plan_set
# удаление из исходных данных проблемных регионов (и тех, у которых суммы не бьются, и тех, кто план не заполнил) и сохранение результата в файл
try:
    bad_mo_region_set = set(df_mo_error["Регион"].tolist())
except Exception:
    bad_mo_region_set = set()
    print("Ошибок МО нет")
try:
    bad_dept_region_set = set(df_dept_error["Регион"].tolist())
except Exception:
    bad_dept_region_set = set()
    print("Ошибок СП нет")

# обьединяем все списики ошибочных регионов: ошибки сумм МО, ошибки сумм СП, регионы с нулевым планом
bad_regions_list = list(
    bad_mo_region_set.union(bad_dept_region_set).union(bad_regions_sum_plan_set)
)
df_ass_success = df_ass.loc[
    ~df_ass["Регион"].isin(bad_regions_list)
]  # успешные регионы - которых нет в списке ошибочных
df_ass_error = df_ass.loc[
    df_ass["Регион"].isin(bad_regions_list)
]  # плохие регионы, которые есть в списке ошибочных
df_ass_success.to_csv(
    result_12_files_path + "/успешные_регионы.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)
print("Данные по успешным регионам сохранены в файл")
df_ass_error.to_csv(
    result_12_files_path + "/регионы_с_ошибками.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)
print("Данные по регионам с ошибками сохранены в файл")

# архивирование
zip_path = shutil.make_archive(result_12_files_path_arch, "zip", result_12_files_path)
# удалнение папки с файлами ФЛК-1,2
shutil.rmtree(result_12_files_path)
print(
    f"Архив с файлами сохранён: {zip_path}, временная папка {result_12_files_path} удалена"
)

# запись всего зоопарка в этот ваш excel

# разъяснения для юзера
df_notation = pd.DataFrame(
    {
        "данные": [
            "текущий статус",
            "дата с... (включая)",
            "дата по... (исключая)",
            "исходник",
            "фильтр дата-статус",
            "регионы без ошибок",
            "регионы с ошибками",
            "несоответствие МО-СП",
            "несоответствие СП-каб",
            "пустая заявка",
            "пустой план (ОШИБКА)",
            "пустой факт (НЕ ошибка)",
            "повторяющиеся МО",
        ],
        "содержание": [
            last_status,
            last_status_date,
            next_status_date,
            "исходные данные как они есть",
            "исходные данные после применения фильтра по текущему статусу и дате последнего статуса",
            "регионы, данные которых не содержат ошибок",
            "регионы, в данных которых выявлены ошибки",
            "данные по регионам, для которых количество заявленного оборудования в разрезе вида оборудования для МО мнее, чем суммарное количество заявленного оборудования в разрезе вида оборудования для СП, входящих в эту МО",
            "аналогичные данные для несоответствия между количеством оборудования, заявленного для СП, и суммарным количеством оборудования, заявленного для кабинетов, входящих в это СП",
            "данные по регионам, которые не указали вообще ничего (если есть хоть что-нибудь в любой позиции, регион сюда не попадёт) - это НЕ ошибка",
            "данные по регионам, у которых в позициях плана ничего не заявлено - это ОШИБКА",
            "данные по регионам, у которых в позициях факта ничего не заявлено - это НЕ ошибка",
            "данные по МО, которые упоминаются в исходной выгрузке более одного раза (такого быть не должно). Данные по таким МО не обрабатываются, регион не попадает в ошибочные и что сним делать, должен решать аналитик",
        ],
    }
)

# создание Pandas Excel writer, используя XlsxWriter в качестве движка
writer = pd.ExcelWriter(result_12_exc_path, engine="xlsxwriter")

# write each DataFrame to a specific sheet
df_notation.to_excel(writer, sheet_name="о чём этот файл", index=False)
df_ass_0.to_excel(writer, sheet_name="исходник", index=False)
df_ass.to_excel(writer, sheet_name="фильтр дата-статус", index=False)
df_ass_success.to_excel(writer, sheet_name="регионы без ошибок", index=False)
df_ass_error.to_excel(writer, sheet_name="регионы с ошибками", index=False)
df_mo_error.to_excel(writer, sheet_name="несоответствие МО-СП", index=False)
df_dept_error.to_excel(writer, sheet_name="несоответствие СП-каб", index=False)
df_ass_erorr_sum.to_excel(writer, sheet_name="пустая заявка (не ошибка)", index=False)
df_ass_mo_plan_zero.to_excel(writer, sheet_name="пустой план (ошибка)", index=False)
df_ass_mo_fact_zero.to_excel(writer, sheet_name="пустой факт (не ошибка)", index=False)
df_mo_count_more_save.to_excel(writer, sheet_name="повторяющиеся МО", index=False)

# close the Pandas Excel writer and output the Excel file
writer.close()
print("Данные сохранены в файл Excel")